# A First (Bad) Attempt

Each frame is a sample <br>
Data is only from Clara's youtube data on exercise 1 (upright row) and 4 (jumping jacks) in data train1.csv and test1.csv

In [23]:
from __future__ import print_function, division
import torch
import pandas as pd
import numpy as np
import os
from skimage import io, transform
import torch.nn as nn
import torch.nn.functional as F

In [24]:
from torch.utils.data import Dataset

class FramesDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.frames = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.frames)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        positions = self.frames.iloc[idx, :]    
        if int(positions[22][-1]) == 1:
            exc_type = 0
        else:
            exc_type = 1
        
        # index 0 is frame number, index 22 is exc type
        positions = torch.tensor(positions[1:22]).double()
        
#         positions = positions.astype('float').reshape(-1, 21)

        if self.transform:
            positions = self.transform(positions)
        return positions, exc_type
    

In [25]:
train = FramesDataset(csv_file = 'data/train1.csv', root_dir='data/')
test = FramesDataset(csv_file = 'data/test1.csv', root_dir='data/')

trainset = torch.utils.data.DataLoader(train, batch_size=5, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=5, shuffle=True)

In [26]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(21, 48)
        self.fc3 = nn.Linear(48, 2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc3(x)
#         return x
        return F.log_softmax(x, dim=1)   # taking log_softmax not needed, for optimization?

net = Net().double()

In [27]:
import torch.optim as optim

# algorithm for first-order gradient-based optimization, built into PyTorch.
optimizer = optim.Adam(net.parameters(), lr=0.001)  
# optimizer = optim.SGD(net.parameters(), lr=0.01)

In [28]:
# for i, data in enumerate(trainset, 0):
#     x, y = data
# print(data)

In [29]:
EPOCHS = 3

for epoch in range(EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainset, 0):
        # get the inputs; data is dictionary
        X, y = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = net(X)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()

        # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0

print('Finished Training')

Finished Training


In [30]:
correct = 0
total = 0
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X)
#         print(X)
#         print(y)
        for index, i in enumerate(output):
            if torch.argmax(i) == y[index]:
                correct += 1
            total += 1
#             print(torch.argmax(i))
#             print(y[index])
#             print("====================")

In [31]:
print("Accuracy: ", round(correct/total, 3))

Accuracy:  1.0
